# Scope

Goal 1: Use a two-sample t-test to determine if there is a significant change in my average running pace since March 20, 2022. The data will be imported from my Garmin account, which I use to track my runs. The results may help me determine the reason for my chronic fatigue since that time.

Goal 2: Use linear regression model to predict mile pace based on total distance and weather conditions.

In [1]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Import data
activities_all = pd.read_csv('Activities.csv')

## Data Inspection and Cleaning

In [8]:
# Display all columns
pd.set_option('display.max_columns', None)

#View first 5 rows of data
activities_all.head()

,Activity Type,Date,Title,Distance,Calories,Time,Avg HR,Max HR,Avg Run Cadence,Max Run Cadence,Avg Pace,Best Pace,Total Ascent,Total Descent,Avg Stride Length,Moving Time,Elapsed Time,Min Elevation,Max Elevation
0,Running,2022-09-21 17:52:41,Lenexa Running,3.00,274,00:26:48,161,173,167,172,8:55,8:24,81,102,1.08,00:26:44,00:27:26,951,996
1,Running,2022-09-12 16:50:18,Lenexa Running,3.01,281,00:26:21,166,179,168,174,8:46,8:16,60,65,1.09,00:26:19,00:26:25,952,996
2,Running,2022-09-09 16:13:22,Lenexa Running,4.00,417,00:39:06,166,181,156,179,9:46,8:17,149,144,1.06,00:39:06,00:39:06,952,"1,027"
3,Running,2022-09-06 17:03:55,Lenexa Running,3.00,280,00:26:40,163,177,166,172,8:53,8:13,50,61,1.09,00:26:40,00:26:40,952,997
4,Running,2022-08-31 17:30:04,Lenexa Running,3.00,282,00:26:10,167,181,168,172,8:43,8:11,50,54,1.10,00:26:04,00:26:38,952,996


In [5]:
activities_all.columns

Index(['Activity Type', 'Date', 'Favorite', 'Title', 'Distance', 'Calories',
       'Time', 'Avg HR', 'Max HR', 'Avg Run Cadence', 'Max Run Cadence',
       'Avg Pace', 'Best Pace', 'Total Ascent', 'Total Descent',
       'Avg Stride Length', 'Avg Vertical Ratio', 'Avg Vertical Oscillation',
       'Avg Ground Contact Time', 'Avg Run Cadence.1', 'Max Run Cadence.1',
       'Training Stress Score®', 'Avg Power', 'Max Power', 'Grit', 'Flow',
       'Avg. Swolf', 'Avg Stroke Rate', 'Total Reps', 'Dive Time', 'Min Temp',
       'Surface Interval', 'Decompression', 'Best Lap Time', 'Number of Laps',
       'Max Temp', 'Moving Time', 'Elapsed Time', 'Min Elevation',
       'Max Elevation'],
      dtype='object')

In [6]:
# Drop unnecessary columns
activities_all = activities_all[['Activity Type', 'Date', 'Title', 'Distance', 'Calories', 'Time', 'Avg HR', 'Max HR', 'Avg Run Cadence', 'Max Run Cadence', 'Avg Pace','Best Pace', 'Total Ascent', 'Total Descent', 'Avg Stride Length', 'Moving Time', 'Elapsed Time', 'Min Elevation', 'Max Elevation']]

In [14]:
# Rename columns
activities_all.columns = ['activity_type', 'date', 'title', 'distance', 'calories', 'time', 'avg_hr', 'max_hr', 'avg_cadence', 'max_cadence', 'avg_pace', 'best_pace', 'total_ascent', 'total_descent', 'avg_stride', 'moving_time', 'elapsed_time', 'min_elevation', 'max_elevation']

In [15]:
activities_all.head()

,activity_type,date,title,distance,calories,time,avg_hr,max_hr,avg_cadence,max_cadence,avg_pace,best_pace,total_ascent,total_descent,avg_stride,moving_time,elapsed_time,min_elevation,max_elevation
0,Running,2022-09-21 17:52:41,Lenexa Running,3.00,274,00:26:48,161,173,167,172,8:55,8:24,81,102,1.08,00:26:44,00:27:26,951,996
1,Running,2022-09-12 16:50:18,Lenexa Running,3.01,281,00:26:21,166,179,168,174,8:46,8:16,60,65,1.09,00:26:19,00:26:25,952,996
2,Running,2022-09-09 16:13:22,Lenexa Running,4.00,417,00:39:06,166,181,156,179,9:46,8:17,149,144,1.06,00:39:06,00:39:06,952,"1,027"
3,Running,2022-09-06 17:03:55,Lenexa Running,3.00,280,00:26:40,163,177,166,172,8:53,8:13,50,61,1.09,00:26:40,00:26:40,952,997
4,Running,2022-08-31 17:30:04,Lenexa Running,3.00,282,00:26:10,167,181,168,172,8:43,8:11,50,54,1.10,00:26:04,00:26:38,952,996


In [18]:
# Add weather data from Garmin Connect (no function to export with run data) - up to 07/29/2017
activities_all['temp'] = [86, 75, 82, 88, 82, 84, 72, 88, 82, 79, 81, 90, 90, 88, 84, 75, 93, 93, 88, 88, 91, 99, 95, 90, 82, 79, 82, 82, 64, 70, 79, 79, 88, 63, 73, 59, 79, 55, 66, 61, 55, 54, 73, 55, 43, 81, 63, 63, 50, 54, 64, 21, 52, 43, 52, 55, 63, 66, 54, 66, 64, 70, 46, 50, 66, 68, 72, 73, 66, 82, 73, 72, 79, 90, 77, 73, 73, 84, 82, 84, 88, 86, 84, 72, 93, 81, 82, 95, 90, 86, 84, 82, 77, 91, 86, 86, 73, 86, 79, 84, 93, 88, 72, 73, 72, 70, 63, 61, 63, 70, 54, 57, 61, 75, 61, 61, 48, 73, 70, 55, 70, 52, 68, 61, 57, 37, 50, 46, 48, 37, 45, 36, 64, 57, 37, 43, 54, 50, 39, 55, 63, 48, 72, 75, 66, 63, 46, 46, 79, 81, 63, 73, 66, 79, 79, 84, 93, 82, 84, 88, 84, 81, 73, 81, 93, 82, 81, 90, 91, 88, 84, 88, 90, 88, 68, 66, 48, 57, 63, 77, 64, 70, 54, 48, 77, 72, 68, 64, 48, 48, 66, 55, 57, 54, 43, 57, 41, 36, 43, 64, 30, 55, 45, 34, 25, 41, 36, 57, 64, 61, 39, 63, 55, 37, 55, 59, 57, 72, 64, 72, 68, 61, 84, 79, 75, 77, 82, 82, 66, 81, 82, 66, 81, 84, 72, 86, 84, 72, 79, 82, 88, 64, 84, 79, 81, 77, 82, 73, 88, 82, 90, 73, 70, 90, 90, 81, 91, 84, 79, 73, 79, 75, 81, 77, 79, 88, 84, 68, 82, 48, 63, 61, 63, 77, 73, 75, 64, 55, 54, 57, 52, 54, 43, 36, 43, 28, 37, N/A, 30, N/A, N/A, 32, 28, 37, 50, 50, 45, 28, 59, 45, 34, 45, 55, 70, 55, 61, 70, 61, 64, 61, 45, 48, N/A, 82, 88, 77, 72, 86, 79, 79, 81, 90, 75, 81, 82, 88, 77, 95, 75, 79, 88, 91, 88, 88, 84, 81, 95, 99, 86, 93, 93, 84, 93, 84, 72, N/A, 81, 84, 86, 90, 88, 81, 88, 91, 84, N/A, N/A, N/A, N/A, N/A, 84, 73, 81, 81, 84, 84, 61, 81, 81, 73, 75, 63, 66, 50, 61, 46, 43, 55, 45, 61, 54, 64, 52, 61, 54, 59, 61, 50, 46, 36, 48, 30, 39, 41, 45, 45, 70, 57, 66, 52, 50, 45, 52, 48, 48, 64, 68, 63, 77, N/A, N/A, 73, 75, 64, 72, 86, 70, 79, 82, 79, 72, 81, 81, 79, 77, 84, 84, 79, 77, 73, 75, 86, 73, ]
activities_all['humidity'] = [31, 69, 30, 36, 55, 40, 78, 43, 48, 65, 51, 46, 34, 43, 62, 74, 36, 28, 63, 43, 34, 40, 44, 59, 58, 48, 58, 58, 88, 60, 39, 61, 55, 68, 36, 39, 54, 28, 37, 36, 33, 32, 33, 62, 39, 16, 20, 59, 25, 26, 18, 32, 30, 49, 50, 38, 68, 56, 22, 68, 56, 46, 46, 76, 46, 43, 38, 41, 49, 48, 61, 65, 70, 30, 41, 29, 36, 52, 27, 55, 59, 59, 55, 55, 50, 37, 42, 47, 41, 40, 45, 66, 61, 53, 46, 38, 94, 55, 28, 55, 28, 55, 61, 73, 83, 49, 27, 45, 27, 60, 47, 36, 31, 47, 59, 55, 87, 41, 26, 33, 18, 28, 23, 29, 41, 65, 58, 46, 46, 56, 66, 51, 28, 24, 41, 75, 41, 32, 35, 35, 29, 32, 57, 31, 32, 42, 66, 53, 51, 37, 36, 27, 40, 48, 48, 62, 41, 42, 40, 55, 66, 79, 44, 74, 47, 62, 70, 43, 49, 55, 37, 43, 41, 63, 68, 83, 50, 36, 48, 47, 42, 46, 32, 27, 57, 38, 28, 83, 87, 71, 46, 16, 36, 41, 42, 67, 45, 55, 70, 37, 64, 33, 53, 69, 80, 48, 55, 41, 64, 39, 41, 52, 38, 93, 58, 31, 44, 38, 37, 43, 40, 72, 66, 54, 74, 47, 45, 37, 88, 62, 70, 100, 70, 43, 94, 59, 62, 83, 74, 55, 36, 88, 45, 45, 45, 83, 58, 89, 46, 55, 52, 89, 94, 52, 52, 79, 53, 52, 45, 78, 70, 65, 62, 65, 23, 36, 55, 68, 55, 93, 83, 72, 72, 54, 41, 39, 68, 38, 67, 41, 35, 44, 81, 69, 56, 74, 52, N/A, 69, N/A, N/A, 80, 100, 44, 34, 46, 66, 74, 41, 57, 47, 57, 51, 40, 67, 39, 46, 77, 45, 39, 57, 87, N/A, 66, 52, 79, 83, 49, 54, 54, 74, 59, 69, 39, 48, 52, 69, 39, 54, 65, 33, 28, 33, 46, 62, 84, 39, 33, 25, 44, 50, 55, 56, 48, 73, N/A, 70, 52, 59, 49, 43, 42, 46, 41, 52, N/A, N/A, N/A, N/A, N/A, 52, 61, 62, 70, 43, 29, 72, 62, 32, 25, 22, 25, 40, 40, 39, 66, 36, 38, 76, 55, 47, 18, 32, 31, 19, 23, 22, 34, 13, 35, 32, 43, 87, 56, 39, 24, 60, 36, 49, 50, 76, 57, 71, 37, 27, 42, 40, 55, 39, N/A, N/A, 78, 41, 83, 41, 59, 73, 51, 48, 51, 78, 51, 42, 61, 94, 66, 62, 65, 65, 78, 44, 46, 78, ]
activities_all['wind'] = [8, 21, 12, 7, 7, 5, 18, 5, 5, 12, 6, 12, 9, 5, 0, 8, 5, 6, 15, 7, 13, 15, 9, 18, 7, 15, 9, 20, 8, 16, 6, 10, 21, 6, 17, 13, 23, 18, 9, 13, 16, 20, 16, 8, 7, 12, 6, 7, 5, 17, 16, 12, 9, 9, 14, 14, 14, 20, 15, 10, 7, 3, 3, 0, 5, 12, 12, 7, 9, 10, 8, 8, 7, 9, 9, 9, 10, 10, 9, 9, 7, 9, 8, 0, 16, 7, 8, 5, 8, 5, 7, 7, 6, 14, 9, 6, 8, 15, 15, 6, 8, 6, 3, 12, 14, 10, 10, 5, 14, 13, 13, 13, 12, 23, 15, 13, 14, 22, 18, 5, 10, 10, 18, 17, 17, 5, 13, 10, 10, 7, 9, 3, 20, 14, 5, 12, 8, 14, 12, 12, 7, 10, 12, 15, 13, 7, 9, 7, 14, 6, N/A, 21, 12, 10, 9, 12, 13, 6, 17, 9, 6, 9, 6, 10, 12, 0, 14, 13, 12, 12, 7, 14, 12, 16, 9, 8, 6, 14, 12, 9, 12, 12, 7, 27, 14, 17, 12, 0, 7, 8, 6, 3, 6, 6, 8, 9, 8, 13, 15, 8, 15, 13, 8, 8, 7, 13, 5, 12, 22, 7, 12, 7, N/A, 10, 6, 17, 24, 16, 17, 13, 0, 7, 15, 8, 16, 7, 3, 7, 9, 12, 7, 16, 15, 10, 5, 9, 7, 7, 5, 13, 14, 5, 10, 7, 18, 12, 12, 0, 0, 14, 10, 0, 6, 8, 10, 8, 12, 10, 12, 13, 14, 12, 8, 15, 9, 10, 16, 9, 12, 0, 3, 15, 0, 16, 18, 14, 6, 12, 7, 18, 12, 6, 9, 6, 10, 13, 15, N/A, 13, N/A, N/A, 15, 13, 12, 17, 5, 12, 13, 16, 12, 5, 10, 8, 14, 8, 7, 7, 10, 8, 6, 6, 0, N/A, 14, 25, 15, 13, 5, 12, 6, 13, 17, 10, 6, 14, 12, 5, 12, 9, 9, 7, 5, 8, 10, 9, 7, 8, 5, 7, 9, 14, 8, 12, 14, 10, N/A, 6, 9, 0, 10, 6, 10, 13, 9, 12, N/A, N/A, N/A, N/A, N/A, 10, 3, 10, 7, 10, 6, 0, 16, 28, 14, 13, 13, 10, 3, 16, 12, 12, 6, 12, 17, N/A, 21, 8, 3, 24, 20, 10, 7, 13, 13, 23, 3, 13, 7, 14, 12, 16, 12, 16, 8, 8, 12, 17, 20, 10, 5, 12, 0, 7, N/A, N/A, 10, 10, 3, 10, 13, 7, 0, 9, 10, 10, 13, 6, 13, 5, 8, 0, 7, 8, 9, 3, 5, 5, ]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   activity_type  494 non-null    object 
 1   date           494 non-null    object 
 2   title          494 non-null    object 
 3   distance       494 non-null    float64
 4   calories       494 non-null    object 
 5   time           494 non-null    object 
 6   avg_hr         494 non-null    object 
 7   max_hr         494 non-null    object 
 8   avg_cadence    494 non-null    int64  
 9   max_cadence    494 non-null    int64  
 10  avg_pace       494 non-null    object 
 11  best_pace      494 non-null    object 
 12  total_ascent   494 non-null    object 
 13  total_descent  494 non-null    object 
 14  avg_stride     494 non-null    float64
 15  moving_time    494 non-null    object 
 16  elapsed_time   494 non-null    object 
 17  min_elevation  494 non-null    object 
 18  max_elevat